<a href="https://colab.research.google.com/github/Naman-chopra/Kheti_baadi/blob/main/SPH_TRAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
path="/content/drive/MyDrive/SPH"

In [12]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
from sklearn import *
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10,10)})



In [19]:
df=pd.read_csv(path+"/dataset.csv")
print(df.describe())
print(df.isna().sum())
df.label.value_counts()

                N           P           K          ph          EC           S  \
count  620.000000  620.000000  620.000000  620.000000  620.000000  620.000000   
mean   135.701613   52.508065  161.401613    6.106290    0.885468    0.484085   
std     31.808083   24.673538   81.285780    0.693782    0.637214    5.775388   
min     30.000000   10.000000   51.000000    4.500000    0.010000    0.011100   
25%    116.000000   30.000000   89.000000    5.700000    0.297500    0.086000   
50%    138.000000   54.000000  157.500000    6.100000    0.780000    0.168000   
75%    160.000000   75.000000  206.000000    6.500000    1.390000    0.240000   
max    198.000000  100.000000  369.000000    8.000000    2.480000  113.200000   

               Cu          Fe           Mn          Zn           B  
count  620.000000  620.000000   620.000000  620.000000  620.000000  
mean    17.315452  117.212500   219.905887   34.340839   28.750710  
std      6.441150   62.775199   341.998793   11.543871   19.636

pomegranate    104
mango          104
grapes         104
mulberry       104
ragi           104
potato         100
Name: label, dtype: int64

In [ ]:
df1=pd.read_csv(path+"/Crop_recommendation.csv")
print(df1.describe())
print(df1.isna().sum())
df1.label.value_counts()

In [18]:
X=df.drop(['N','p','K'],axis=1)
Y=df.label


#2. GRID SEARCH

##2.1 Linear Reg

In [ ]:
from sklearn.linear_model import LinearRegression as LR

grid_params={'fit_intercept':[True,False],'copy_X':[True],'n_jobs':[-1]}
gs=GridSearchCV(LR(),grid_params,verbose=3,cv=10,n_jobs=1,scoring='neg_mean_squared_error')
gs_results=gs.fit(X,Y)
print(gs_results.best_score_)
print(gs_results.best_params_)
lin_reg=gs_results.best_estimator_

# -0.8873596762931012
# {'copy_X': True, 'fit_intercept': True, 'n_jobs': -1}

Fitting 10 folds for each of 2 candidates, totalling 20 fits
-0.8873596762931012
{'copy_X': True, 'fit_intercept': True, 'n_jobs': -1}


##2.2 Polynomial

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
x_poly=poly.fit_transform(X)
grid_params={'fit_intercept':[True,False],'copy_X':[True],'n_jobs':[-1]}
gs=GridSearchCV(LR(),grid_params,verbose=3,cv=10,n_jobs=1,scoring='neg_mean_squared_error')
gs_results=gs.fit(x_poly,Y)
print(gs_results.best_score_)
print(gs_results.best_params_)
poly_lin_reg=gs_results.best_estimator_

Fitting 10 folds for each of 2 candidates, totalling 20 fits
-18.403777128952377
{'copy_X': True, 'fit_intercept': False, 'n_jobs': -1}


##2.3 Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

grid_params={'alpha':[0.001,0.01,0.1,1,10,100],'max_iter':[50,100,500,1000,2000,10000],'solver':['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],'fit_intercept':[True,False],'copy_X':[True]}
gs=GridSearchCV(Ridge(),grid_params,verbose=3,cv=5,n_jobs=1,scoring='neg_mean_squared_error')
gs_results=gs.fit(X,Y)
print(gs_results.best_score_)
print(gs_results.best_params_)
ridge_reg=gs_results.best_estimator_

# -0.8745833373823224
# {'alpha': 10, 'copy_X': True, 'fit_intercept': True, 'max_iter': 50, 'solver': 'saga'}
# /usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
#   ConvergenceWarning,


Fitting 5 folds for each of 504 candidates, totalling 2520 fits
-0.8745833373823224
{'alpha': 10, 'copy_X': True, 'fit_intercept': True, 'max_iter': 50, 'solver': 'saga'}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


##2.4 Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

grid_params={'alpha':[0.001,0.01,0.1,1,10,100],'max_iter':[50,100,500,1000,2000,10000],
             'fit_intercept':[True,False],'copy_X':[True],
             'warm_start':[True,False],
             'selection':['cyclic','random']
             }
gs=GridSearchCV(Lasso(),grid_params,verbose=3,cv=5,n_jobs=1,scoring='neg_mean_squared_error')
gs_results=gs.fit(X,Y)
print(gs_results.best_score_)
print(gs_results.best_params_)
lasso_reg=gs_results.best_estimator_

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
-0.8740852299107653
{'alpha': 0.001, 'copy_X': True, 'fit_intercept': True, 'max_iter': 50, 'selection': 'random', 'warm_start': True}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.761e+02, tolerance: 8.662e-01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


##2.5 SVR

In [ ]:
from sklearn.svm import SVR


grid_params = {'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),'C' : [1,5,10,50],'degree' : [2,3,8],'coef0' : [0.01,10,0.5],'gamma' : ('auto','scale')},

gs=GridSearchCV(SVR(),grid_params,verbose=3,cv=5,n_jobs=1,scoring='neg_mean_squared_error')
gs_results=gs.fit(X,Y)
# print(gs_results.best_score_)
print(gs_results.best_params_)
svr_reg=gs_results.best_estimator_

##2.6 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor as Rf

grid_params={'n_estimators':[100,500],
             'criterion':['squared_error'],
             'max_features':['sqrt',None],
             'n_jobs':[-1],
             'verbose':[3],
             'warm_start':[True,False]
             }

gs=GridSearchCV(Rf(),grid_params,verbose=3,cv=5,n_jobs=1,scoring='neg_mean_squared_error')
gs_results=gs.fit(X,Y)
print(gs_results.best_score_)
print(gs_results.best_params_)
rf_reg=gs_results.best_estimator_
# -0.7481391830687684
# {'criterion': 'squared_error', 'max_features': None, 'n_estimators': 500, 'n_jobs': -1, 'verbose': 3, 'warm_start': False}


In [ ]:
from sklearn.ensemble import RandomForestRegressor as Rf

rf_reg=Rf(criterion='squared_error',n_estimators=500,warm_start=False)


RandomForestRegressor(n_estimators=500)

##2.7 GBR

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor as GBR


grid_params= {
    "loss":["squared_error"],
    "learning_rate": [0.01,0.1,1,10],
    # "min_samples_split": np.linspace(1, 5,10 ),
    # "min_samples_leaf": np.linspace(1, 5,10),
    "max_depth":[10],
    # "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mse",'squared_error'],
    # "subsample":[0.2,0.5, 0.7, 0.9, 1.0],
    "n_estimators":[100,500]
    }

gs=GridSearchCV(GBR(),grid_params,verbose=3,cv=5,n_jobs=1,scoring='neg_mean_squared_error')
gs_results=gs.fit(X,Y)
print(gs_results.best_score_)
print(gs_results.best_params_)
gbr_reg=gs_results.best_estimator_


# -0.8444609546465334
# {'criterion': 'mse', 'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 10, 'n_estimators': 500}
